### Comparison of results from validate_drp and SPORK on the same dataset.

In particular, this comparison is run on HSC RC2.

In [ ]:
import random
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.table import hstack, Column
import matplotlib.pyplot as plt
import json
import os

import lsst.verify
import lsst.geom as geom
import lsst.daf.persistence as dafPersist
from lsst.verify.gen2tasks import register, MetricsControllerTask
from lsst.verify.tasks import MetricTask
import lsst.daf.butler as dafButler

In [ ]:
# Output from a run of validate_drp:
validate_job_g9615 = '/project/jcarlin/mvm_w_25/9615/matchedVisit_HSC-G.json'
validate_job_r9615 = '/project/jcarlin/mvm_w_25/9615/matchedVisit_HSC-R.json'
validate_job_i9615 = '/project/jcarlin/mvm_w_25/9615/matchedVisit_HSC-I.json'

The validate_drp results come from executing the following commands on the v20_0_0_rc1 processing of the RC2 dataset:

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9615 filter='HSC-G^HSC-R^HSC-I'

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25/9813 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9813 filter='HSC-G^HSC-R^HSC-I'

matchedVisitMetrics.py /datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349 --output /project/jcarlin/mvm_w_25/9697 --config instrumentName='HSC' datasetName='HSC-RC2' --id tract=9697 filter='HSC-G^HSC-R^HSC-I'

In [ ]:
# Read in the .json saved by each validate_drp run:
with open(validate_job_g9615) as f:
    job_g9615 = lsst.verify.Job.deserialize(**json.load(f))
with open(validate_job_r9615) as f:
    job_r9615 = lsst.verify.Job.deserialize(**json.load(f))
with open(validate_job_i9615) as f:
    job_i9615 = lsst.verify.Job.deserialize(**json.load(f))

In [ ]:
# Show a metric report in the notebook (use "spec_tags" to specify design, stretch, or minimum req level):
job.report(spec_tags=['design']).show()

In [ ]:
# Extract the measurements from the JSON:
meas_g9615 = job_g9615.measurements.json
meas_r9615 = job_r9615.measurements.json
meas_i9615 = job_i9615.measurements.json

In [ ]:
# Compile the measurements into arrays:

gen2_names_g9615 = []
gen2_vals_g9615 = []
gen2_units_g9615 = []

for met in meas_g9615:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_g9615.append(met['metric'])
    gen2_vals_g9615.append(met['value'])
    gen2_units_g9615.append(met['unit'])
    
gen2_names_r9615 = []
gen2_vals_r9615 = []
gen2_units_r9615 = []

for met in meas_r9615:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_r9615.append(met['metric'])
    gen2_vals_r9615.append(met['value'])
    gen2_units_r9615.append(met['unit'])
    
gen2_names_i9615 = []
gen2_vals_i9615 = []
gen2_units_i9615 = []

for met in meas_i9615:
    # print(met['metric'], met['value'], met['unit'])
    gen2_names_i9615.append(met['metric'])
    gen2_vals_i9615.append(met['value'])
    gen2_units_i9615.append(met['unit'])

In [ ]:
# Make the arrays into astropy tables:
tab_gen2_g9615 = Table([gen2_names_g9615, gen2_vals_g9615, gen2_units_g9615], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))
tab_gen2_r9615 = Table([gen2_names_r9615, gen2_vals_r9615, gen2_units_r9615], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))
tab_gen2_i9615 = Table([gen2_names_i9615, gen2_vals_i9615, gen2_units_i9615], names=['metric', 'value', 'units'], dtype=(str, 'f2', str))

### Now get the Gen 3 butler results from running SPORK on RC2

In [ ]:
try: butler_gen3 = dafButler.Butler(config='/home/jcarlin/rc2v20_butler.yaml')
except ValueError as e: print(e)

In [ ]:
registry = butler_gen3.registry

In [ ]:
# To see what collections are in the repo:
#for c in registry.queryCollections():
#    print(c)

In [ ]:
# To see what dataset types are (potentially) available:
#for x in registry.queryDatasetTypes():
#    print(x)

In [ ]:
# These are the names of the metrics we plan to extract from the repo:
metricnames = ['PA1', 'PA2_design_gri', 'PF1_design_gri', 'AM1', 'AM2', 'AM3', 'AD1_design', 'AF1_design',
               'AD2_design', 'AF2_design', 'AD3_design', 'AF3_design', 'TE1', 'TE2', 'AB1_design']

In [ ]:
collectionName = 'kbechtol/test_metrics_4'

In [ ]:
def get_metric_meas(collectionName, filt, tract, metrics):
    
    metric_agg_vals = []
    metric_unit = []
    metric_filter = []
    metric_name = []
    metric_ids = []
    metric_meas = []

    did={'tract':str(tract), 'band':filt, 'instrument':'HSC', 'skymap':'hsc_rings_v1'}

    for metric in metrics:
        dataset = f"metricvalue_summary_validate_drp_{metric}"
        query = registry.queryDatasets(datasetType=dataset, collections=[collectionName], dataId=did)
        for meas in query:            
            metric_tmp = butler_gen3.get(meas, collections=[collectionName])
            metric_agg_vals.append(metric_tmp.quantity.value)
            metric_unit.append(metric_tmp.quantity.unit)
            metric_name.append(metric)
            metric_ids.append(meas)
            metric_meas.append(metric_tmp)
            metric_filter.append(meas.dataId['band'])
    
    tab_metrics = Table([metric_name, metric_unit, metric_filter, metric_agg_vals], 
                        names=('metric', 'unit', 'filter', str(tract)+'_gen3'), dtype=(str, str, str, 'f2'))
        
    return(tab_metrics)

In [ ]:
# Extract astropy tables of the metric values by calling the function defined above:
tab_gen3_g9615 = get_metric_meas(collectionName, 'g', 9615, metricnames)
tab_gen3_r9615 = get_metric_meas(collectionName, 'r', 9615, metricnames)
tab_gen3_i9615 = get_metric_meas(collectionName, 'i', 9615, metricnames)

In [ ]:
tab_gen3_g9813

Loop over the names of the metrics, picking out the corresponding measurements from Gen2 and Gen3 (and possibly from tracts/filters separately), compiling them into a table together. (Later, we may want to add in the specs?)

In [ ]:
tab_metrics_g9615 = tab_gen3_g9615.copy()
matchcol=np.zeros(len(tab_metrics_g9615['9615_gen3']))

for i in range(len(tab_metrics_g9615['metric'])):
    name = tab_metrics_g9615['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_g9615['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_g9615['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_g9615_all = hstack([tab_metrics_g9615, Column(matchcol, name='9615_gen2', dtype='f2')])

In [ ]:
tab_metrics_r9615 = tab_gen3_r9615.copy()
matchcol=np.zeros(len(tab_metrics_r9615['9615_gen3']))

for i in range(len(tab_metrics_r9615['metric'])):
    name = tab_metrics_r9615['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_r9615['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_r9615['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_r9615_all = hstack([tab_metrics_r9615, Column(matchcol, name='9615_gen2', dtype='f2')])

In [ ]:
tab_metrics_i9615 = tab_gen3_i9615.copy()
matchcol=np.zeros(len(tab_metrics_i9615['9615_gen3']))

for i in range(len(tab_metrics_i9615['metric'])):
    name = tab_metrics_i9615['metric'][i]
    find_in_tab2 = np.where(np.char.find(tab_gen2_i9615['metric'], name) >= 0)

    if np.size(find_in_tab2) > 0:
        matchcol[i] = tab_gen2_i9615['value'][find_in_tab2[0]]
    else:
        matchcol[i] = np.nan
    
tab_metrics_i9615_all = hstack([tab_metrics_i9615, Column(matchcol, name='9615_gen2', dtype='f2')])

In [ ]:
# Combine them all into a single table:
tab_gr_tmp = hstack([tab_metrics_g9615_all, tab_metrics_r9615_all['filter','9615_gen3', '9615_gen2']])
tab_metrics_9615_all = hstack([tab_gr_tmp, tab_metrics_i9615_all['filter','9615_gen3', '9615_gen2']])

In [ ]:
tab_metrics_9615_all

In [ ]:
# Write out a Latex table
tab_metrics_9615_all.write('compare_metrics_gen2and3_tract9615.tex', format='ascii.latex')

### Plot the metric measurements per patch

In [ ]:
dimensions = registry.queryDimensions('patch',
                                      dataId={'tract':9615, 'band':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1'},
                                      collections=[collectionName],datasets='metricvalue_validate_drp_PA1')

In [ ]:
patch_tmp = []
for d in dimensions:
    patch_tmp.append(d['patch'])

In [ ]:
patch = np.unique(patch_tmp)

In [ ]:
patch

In [ ]:
metric_agg_vals = np.zeros((len(patch),len(metricnames)))
metric_agg_vals[:] = np.nan
metric_patch = []
metric_ra = []
metric_dec = []

for p in patch:
    metric_patch.append(p)
    did={'tract':9615, 'band':'r', 'instrument':'HSC', 'skymap':'hsc_rings_v1', 'patch':p}
    
    for i in range(len(metricnames)):
        metric = metricnames[i]
        dataset = f"metricvalue_validate_drp_{metric}"
    
        query = registry.queryDatasets(datasetType=dataset, collections=[collectionName], dataId=did)
        for meas in query:            
            metric_tmp = butler_gen3.get(meas, collections=[collectionName])
            metric_agg_vals[p,i]=metric_tmp.quantity.value

            if i == 0:
                coadd = butler_gen3.get('deepCoadd', collections=['RC2/v20_0_0_rc1'],dataId=did)
                wcs = coadd.getWcs()
                xy0 = wcs.pixelToSky(geom.Point2D(coadd.getXY0()))
                metric_ra.append(xy0.getRa().asDegrees())
                metric_dec.append(xy0.getDec().asDegrees())
    
tab_metrics_perpatch = Table([metric_patch, metric_ra, metric_dec, metric_agg_vals],
                             names=('patch', 'ra', 'dec', 'metrics'))

In [ ]:
params = {
   'axes.labelsize': 28,
   'font.size': 24,
   'legend.fontsize': 14,
   'xtick.major.width': 3,
   'xtick.minor.width': 2,
   'xtick.major.size': 12,
   'xtick.minor.size': 6,
   'xtick.direction': 'in',
   'xtick.top': True,
   'lines.linewidth':3,
   'axes.linewidth':3,
   'axes.labelweight':3,
   'axes.titleweight':3,
   'ytick.major.width':3,
   'ytick.minor.width':2,
   'ytick.major.size': 12,
   'ytick.minor.size': 6,
   'ytick.direction': 'in',
   'ytick.right': True,
   'figure.figsize': [10, 8]
   }

plt.rcParams.update(params)

metric_to_plot = 'AD1'
find_metric_name = np.where(np.char.find(metricnames, metric_to_plot) >= 0)

plt.scatter(tab_metrics_perpatch['ra'], tab_metrics_perpatch['dec'], c=tab_metrics_perpatch['metrics'][:,find_metric_name], 
            cmap='rainbow', s=60)
#plt.xlim(216.4,216.1)
#plt.ylim(0.742,0.746)
plt.gca().invert_xaxis()
plt.xlabel(r'RA (deg)')
plt.ylabel(r'Dec (deg)')
plt.minorticks_on()
plt.colorbar(label=metricnames[find_metric_name[0][0]])
plt.show()